In [131]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [116]:
import json
import pandas as pd
import random
p= .1#% of the lines
#keep the header, then take only 1% of lines
#if random from [0,1] interval is greater than 0.01 the row will be skipped
raw_data_with_measures = pd.read_csv(
         '/content/data.csv', 
         skiprows=lambda i: i>0 and random.random() > p)

In [46]:
 #create subdataframe
# raw_data_with_measures.to_csv('raw_data.csv') #to csv 
# !cp raw_data.csv "drive/My Drive/" #store in drive 

In [ ]:
raw_data_with_measures['artists'] = raw_data_with_measures['artists'].apply(lambda x: x[1:-1].split(', ')) # makes the string lists in artists column into actual lists

raw_data_with_measures = raw_data_with_measures.explode('artists') # opens the lists so that each artist is on a song has his own row with the song

raw_data_with_measures['artists'] = raw_data_with_measures['artists'].apply(lambda x: x.strip("'")) #  we want to take off the quotes on the artists

raw_data =  raw_data_with_measures.loc[:,['artists','id','name','popularity','year']] # we will just focus on these columns for now, and rename our data to raw_data


raw_data

In [ ]:
raw_data_genre = pd.read_csv('data_w_genres.csv')
raw_data_genre.head(4)

In [ ]:
raw_data_genre['genres'] = raw_data_genre['genres'].apply(lambda x: x[1:-1].split(', '))  # makes it into actual list instead of string list

for i in raw_data_genre.index:

    raw_data_genre['genres'].loc[i] = list(filter(None, raw_data_genre['genres'][i]))  # filters out empty values in each list
    

raw_data_genre.head(10)

In [ ]:
missing_genre_artists = raw_data_genre[raw_data_genre['genres'].map(lambda x: len(x)) < 1]
missing_genre_artists

In [ ]:
missing_genre_artists_split = missing_genre_artists.copy()
missing_genre_artists_split['artists'] = missing_genre_artists_split['artists'].apply(lambda x: x.split())  # split into words
missing_genre_artists_split

In [ ]:
# lets find most common words in artists

def word_count(data):

    all_words = []  
    for elmnt in data['artists']:  
        all_words += elmnt  

    val_counts = pd.Series(all_words).value_counts()

    return val_counts.head(40)

word_count(missing_genre_artists_split)

In [ ]:
pd.options.mode.chained_assignment = None 

for i in raw_data_genre.index:
    
    if 'Broadway' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'broadway'")
    if 'Motion Picture' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'movie tunes'")
    if 'Cast' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'ensemble'")     # we will put any cast artists under ensemble genre 
    if 'Ensemble' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'ensemble'")  
    if 'Orchestra' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'orchestra'")
    if 'Orquesta' in raw_data_genre['artists'].loc[i]:              # these are just vara
        raw_data_genre['genres'].loc[i].append("'orchestra'")
    if 'Orchestre' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'orchestra'")
    if 'Chorus' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'chorus'")
    if 'Choir' in raw_data_genre['artists'].loc[i]:
        raw_data_genre['genres'].loc[i].append("'choir'")
        
    raw_data_genre['genres'].loc[i] = list(dict.fromkeys(raw_data_genre['genres'].loc[i])) # removes duplicates
    

raw_data_genre.head(12)

In [ ]:
#create final df that removes artists without genres 
final_data_genre = raw_data_genre[raw_data_genre['genres'].map(lambda x: len(x)) > 0] # removes empty lists
final_data_genre = final_data_genre.reset_index(drop=True)
final_data_genre = final_data_genre.drop('count', axis =1 )
final_data_genre.rename(columns={'popularity': 'Artist Popularity'}, inplace=True)
final_data_genre['Artist Popularity'] = final_data_genre['Artist Popularity'].astype(int)
final_data_genre

In [129]:
merged_df = raw_data.merge(final_data_genre, how = 'inner', on = ['artists'])
merged_df_copy = merged_df.copy()
merged_df_copy.rename(columns={'artists': 'Artist', 'name':'Song Name','popularity':'Song Popularity','year':'Year','genres':'Genres'}, inplace=True)
merged_df_copy['Genres_cleaned'] = [','.join(map(str, l)) for l in merged_df_copy['Genres']]
merged_df_copy.drop(columns=['Genres'], inplace=True)
final_df = merged_df_copy.drop(columns=['key', 'mode'])


In [137]:
final_df.to_csv('final_df.csv') #to csv 
!cp final_df.csv "drive/My Drive/" #store in drive 

In [132]:
#dummy encode genres 
new_df = pd.concat([final_df.drop('Genres_cleaned', 1), final_df['Genres_cleaned'].str.get_dummies(sep=",")], 1)

In [133]:
from sklearn.preprocessing import LabelEncoder
artist_encoder = LabelEncoder()
song_encoder = LabelEncoder()
new_df['Artist'] = artist_encoder.fit_transform(new_df['Artist'])
new_df['Song Name'] = song_encoder.fit_transform(new_df['Song Name'])

In [136]:
new_df.to_csv('new_df.csv') #to csv 
!cp new_df.csv "drive/My Drive/" #store in drive 

In [138]:
from sklearn.neighbors import NearestNeighbors

# Instantiate and fit knn to the correct columns
knn = NearestNeighbors(n_neighbors=1000)
knn.fit(new_df[new_df.columns[5:]])

def predict(track_id):

    obs = new_df.index[new_df['id'] == track_id]
    series = new_df.iloc[obs, 5:].to_numpy()

    neighbors = knn.kneighbors(series)
    new_obs = neighbors[1][0][6:20]
    return list(new_df.loc[new_obs, 'id'])

In [ ]:
predict('6N6tiFZ9vLTSOIxkj8qKrd')

In [139]:
import joblib

joblib.dump(knn, "KNN_final.joblib", compress=2, protocol=None, cache_size=None)


['KNN_final.joblib']